# Analyzing the improved genome assembly

## run the notebook serve on a client node

```
    srun --pty --nodes=1  --ntasks-per-node=1 --cpus-per-task=28 --time 100:00:00 --job-name bash-jupyter bash
    conda activate ont_assembly
    jupyter notebook --ip 0.0.0.0 --port 3001 --no-browser
```

## bokeh imports

In [2]:
from bokeh.plotting import figure, output_file, save
from bokeh.palettes import viridis
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, LabelSet, Whisker
from bokeh.models.tickers import FixedTicker
from bokeh.transform import jitter
from bokeh.layouts import column, row, gridplot
from math import pi
from bokeh.layouts import row
import random
import re
from identify_collapsed_regions import load_dist_dev, load_gaps

COLOR_PALETTE = ["#0072B2", "#D55E00", "#009E73", "#E69F00", "#CC79A7", "#56B4E9", "#F0E442"]

output_notebook()

Loading BokehJS ...

# identify reverse complemented scaffolds

In [4]:
# load the data:


def load_gaps_by_contig(file_name):
    ret = {}

    with open(file_name, "r") as in_file:
        for line in in_file:
            if line[0] == "#":
                continue
            chrom, source, anno_type, start, end, *extra = line.strip().split()
            if chrom not in ret:
                ret[chrom] = []
            start = int(start)
            end = int(end)
            ret[chrom].append((start, end))
    return ret

gap_pos = load_gaps_by_contig("../data/out/7_closed_gaps_b/gaps.gff3")

def load_read_pos_and_str(file_name_in):
    ret = {}
    with open(file_name_in, "r") as file_in:
        for line in file_in:
            if line[0] == "#":
                continue
            read_name, chrom, pos1, pos2, strand1, strand2, map_q = line.strip().split("\t")
            pos1 = int(pos1)
            pos2 = int(pos2)
            map_q = int(map_q)
            strand1 = strand1 == "0"
            strand2 = strand2 == "0"
            if chrom not in ret:
                ret[chrom] = []
            ret[chrom].append((read_name, min(pos1, pos2), max(pos1, pos2), strand1, strand2, map_q))
    return ret

read_pos_and_strand = load_read_pos_and_str("../data/out/7.1_gap_spanning_reads_on_b/read_pos_and_strnd.tsv")

In [5]:
# filter the reads

def filter_reads(reads, gaps):
    ret = {}
    for contig, r in reads.items():
        ret[contig] = []
        if not contig in gaps:
            continue
        gaps[contig].sort()
        r.sort(key=lambda x: x[1])

        for r_name, pos1, pos2, s1, s2, map_q in r:
            for start, end in gaps[contig]:
                if pos1 < end and pos2 > start:
                    ret[contig].append((r_name, pos1, pos2, s1, s2, map_q))
                    break
    return ret

filtered_reads = filter_reads(read_pos_and_strand, gap_pos)

In [10]:
# plot

contigs_to_check = set()
for contig, r in filtered_reads.items():
    for r_name, pos1, pos2, s1, s2, _ in r:
        if s1 != s2:
            contigs_to_check.add(contig)

all_contigs = set()
for contig, r in filtered_reads.items():
    if "Chr" in contig:
        all_contigs.add(contig)


def plot_support(chrom):
    f = figure(x_axis_label="genome position", title=chrom)

    x1s = []
    x2s = []
    ys = []
    cs = []
    for idx, (r_name, pos1, pos2, s1, s2, map_q) in enumerate(filtered_reads[chrom]):
        x1s.append(pos1)
        x2s.append(pos2)
        ys.append(idx * 2)
        cs.append(("blue" if s1 == s2 else "orange") if map_q > 30 else ("lightblue" if s1 == s2 else "yellow"))
    if chrom in gap_pos:
        for start, end in gap_pos[chrom]:
            x1s.append(start)
            x2s.append(end)
            ys.append(-2)
            cs.append("red")

    f.hbar(left=x1s, right=x2s, y=ys, color=cs, line_width=2)
    return f


show(column([plot_support(chrom) for chrom in sorted(list(contigs_to_check))]), notebook_handle=True)

## load the data

In [5]:
ref_names, ref_dev = load_dist_dev("../data/out/10.2_test_gap_spanning_reads/distance_deviation.tsv")
gap_pos = load_gaps("../data/out/10.3_test_annotate_gaps/gaps.gff3")


### print mq>= 30 connections between contigs

In [6]:

contig_connections = {}
with open("../data/out/10.2_test_gap_spanning_reads/contig_connecting_reads.tsv", "r") as in_file:
    for line in in_file.readlines():
        if line[0] == "#":
            continue
        rname, o_rname, pos, o_rpos, rev_stnd, o_rev_stnd, map_q, o_map_q = line[:-1].split()
        ctg1 = min(rname, o_rname)
        ctg2 = max(rname, o_rname)
        if int(map_q) < 30 or int(o_map_q) < 30:
            continue
        if not ctg1 in contig_connections:
            contig_connections[ctg1] = {}
        if not ctg2 in contig_connections[ctg1]:
            contig_connections[ctg1][ctg2] = 0
        contig_connections[ctg1][ctg2] += 1

connection_list = []
for k, v in contig_connections.items():
    for k2, v2 in v.items():
        connection_list.append((v2, k, k2))
connection_list.sort(reverse=True)
for v, k, k2 in connection_list[:20]:
    print(v, k, "->", k2)

203 Chr11_B_Tb427v10 -> Chr9_B_Tb427v10
151 unitig_148_Tb427v10 -> unitig_2569_Tb427v10
149 Chr1_A_Tb427v10 -> Chr6_A_Tb427v10
147 Chr8_B_Tb427v10 -> unitig_3082_Tb427v10
139 Chr6_A_Tb427v10 -> unitig_252_Tb427v10
128 Chr9_A_Tb427v10 -> Chr9_B_Tb427v10
116 unitig_1852_Tb427v10 -> unitig_1853_Tb427v10
97 unitig_1853_Tb427v10 -> unitig_1897_Tb427v10
90 Chr6_B_Tb427v10 -> unitig_288_Tb427v10
89 Chr1_B_Tb427v10 -> unitig_1853_Tb427v10
83 unitig_2332_Tb427v10 -> unitig_309_Tb427v10
80 Chr11_A_Tb427v10 -> Chr11_B_Tb427v10
79 Chr10_A_Tb427v10 -> Chr10_A_Tb427v10_masked_250001_251000
77 Chr9_B_Tb427v10 -> unitig_228_Tb427v10
69 Chr9_B_Tb427v10 -> unitig_2279_Tb427v10
67 BES14_Tb427v10 -> Chr2_A_Tb427v10
66 unitig_2332_Tb427v10 -> unitig_278_Tb427v10
66 Chr6_A_Tb427v10 -> unitig_1947_Tb427v10
66 Chr6_A_Tb427v10 -> unitig_1889_Tb427v10
64 Chr8_A_Tb427v10 -> Chr8_B_Tb427v10


In [8]:
def analyze_connections(picked_contig_a, picked_contig_b):
    f = figure(x_axis_label=picked_contig_a, y_axis_label=picked_contig_b)

    x1s = []
    y1s = []
    cs = []

    with open("../data/out/10.2_test_gap_spanning_reads/contig_connecting_reads.tsv", "r") as in_file:
        for line in in_file.readlines():
            if line[0] == "#":
                continue
            rname, o_rname, pos, o_rpos, rev_stnd, o_rev_stnd, map_q, o_map_q = line[:-1].split()
            # if int(map_q) < 30 or int(o_map_q) < 30:
            #     continue
            pos = int(pos)
            o_rpos = int(o_rpos)

            if rname == picked_contig_b and o_rname == picked_contig_a:
                o_rname, rname, o_rpos, pos, o_rev_stnd, rev_stnd, o_map_q, map_q = \
                    rname, o_rname, pos, o_rpos, rev_stnd, o_rev_stnd, map_q, o_map_q
            if rname == picked_contig_a and o_rname == picked_contig_b:
                x1s.append(pos)
                y1s.append(o_rpos)
                if rev_stnd == o_rev_stnd and rev_stnd == "False":
                    cs.append("blue")
                elif rev_stnd == o_rev_stnd and rev_stnd == "True":
                    cs.append("black")
                elif rev_stnd == "False":
                    cs.append("orange")
                else:
                    assert rev_stnd == "True"
                    cs.append("green")

    f.x(x=x1s, y=y1s, color=cs)


    x1s = []
    x2s = []
    y1s = []
    y2s = []
    cs = []


    for chr, start, end, _ in gap_pos[0].values():
        if chr + "_Tb427v10" == picked_contig_a:
            x1s.append(start)
            x2s.append(end)
            y2s.append(-1)
            y1s.append(-2)
            cs.append("red")
        if chr + "_Tb427v10" == picked_contig_b:
            y1s.append(start)
            y2s.append(end)
            x2s.append(-1)
            x1s.append(-2)
            cs.append("red")

    f.quad(left=x1s, right=x2s, bottom=y1s, top=y2s, color=cs, line_width=2)


    show(f, notebook_handle=True)

for v, k, k2 in sorted(connection_list[:20], key=lambda x: x[1]):
    if "unitig" in k and "unitig" in k2:
        continue
    if not ("Chr" in k or "Chr" in k2):
        continue
    # if ("Chr10" in k or "Chr10" in k2):# and ("unitig" in k or "unitig" in k2):
    analyze_connections(k, k2)

# identify collapsed regions 

# post process the data

In [4]:
from identify_collapsed_regions import *

f = figure(x_axis_label="genome position", y_axis_label="distance deviation")

chr_to_investigate = "Chr4_A"
xs = []
ys = []
cs = []
for r_name, (distance, expected, chr, pos1, pos2, strnd, map_q) in zip(ref_names, ref_dev):
    if chr_to_investigate in chr: # :1507kbp
        xs.append([min(pos1, pos2), max(pos1, pos2)])
        ys.append([distance, distance])
        cs.append("blue" if map_q > 30 else "lightblue")


data = post_process(ref_names, ref_dev)
clusters = cluster(data, distance_y=50)
clusters = filter_clusters_with_counter_indication(clusters, data)

qxs = []
qxe = []
qys = []
qye = []

f = figure(x_axis_label="genome position", y_axis_label="distance deviation")
for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in clusters:
    if chr_to_investigate in cluster_chr: # :1507kbp
        qxs.append(cluster_start)
        qxe.append(cluster_end)
        qys.append(min(cx[3] for cx in c))
        qye.append(max(cx[3] for cx in c))


f.multi_line(xs=xs, ys=ys, color=cs, alpha=0.5)
f.quad(left=qxs, right=qxe, bottom=qys, top=qye, color="green", alpha=0.9)

show(f, notebook_handle=True)

# find gaps with too much flanking sequence

In [3]:
from identify_collapsed_regions import *

f = figure(x_axis_label="genome position", y_axis_label="distance deviation")

chr_to_investigate = "Chr8_A"
xs = []
ys = []
cs = []
for r_name, (distance, expected, chr, pos1, pos2, strnd, map_q) in zip(ref_names, ref_dev):
    if chr_to_investigate in chr: # :1507kbp
        xs.append([min(pos1, pos2), max(pos1, pos2)])
        ys.append([distance, distance])
        cs.append("blue" if map_q > 30 else "lightblue")


data = post_process(ref_names, ref_dev, min_dev=float("inf"), max_dev=500)
clusters = cluster(data, distance_y=50)
clusters = filter_clusters_with_counter_indication(clusters, data)

qxs = []
qxe = []
qys = []
qye = []

f = figure(x_axis_label="genome position", y_axis_label="distance deviation")
for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in clusters:
    if chr_to_investigate in cluster_chr: # :1507kbp
        qxs.append(cluster_start)
        qxe.append(cluster_end)
        qys.append(min(cx[3] for cx in c))
        qye.append(max(cx[3] for cx in c))


f.multi_line(xs=xs, ys=ys, color=cs, alpha=0.5)
f.quad(left=qxs, right=qxe, bottom=qys, top=qye, color="green", alpha=0.9)

show(f, notebook_handle=True)

# analyze the fixing of collapsed regions

## load the remaining data

In [3]:
ref_names, ref_dev = load_dist_dev("../data/out/2.1_gap_spanning_reads/distance_deviation.tsv")
fixed_names, fixed_dev = load_dist_dev("../data/out/22_vpr_new_genome/distance_deviation.tsv")

gap_spanning_reads = {}

with open("../data/out/22_vpr_new_genome/gap_spanning_reads.tsv", "r") as file_in:
    for line in file_in:
        readname, *gaps = line[:-1].strip().split()
        readname = readname.split("_")[0]
        gap_names = []
        for gap in gaps:
            chrom, start, end = re.split(":|-", gap)
            #chrom = chrom[:-len("_Tb427v10")]
            gap_name = chrom + ":" + str(int(start)//1000) + "kbp"
            gap_names.append(gap_name)
        gap_spanning_reads[readname] = " ".join(gap_names)

print(gap_spanning_reads)
print(fixed_names[:10])
print(ref_names[:10])

{'ddfef19b-abf2-4826-98d9-da313467b7e7': 'Chr5_A:888kbp', '81d2e85a-ffc3-4c89-a448-d59ec32747f0': 'Chr1_B:1478kbp', 'bedfc235-3c97-4952-8b41-6bfcd6d650e3': 'Chr4_B:114kbp', '62d39d2f-805a-4ae3-8c66-4a383d6c4379': 'Chr7_B:2003kbp', 'd5d662ba-16f9-405d-be8a-f2cf9330a262': 'Chr1_A:990kbp', '4e69e73a-34a6-4755-8486-cb5168698da8': 'BES14:60kbp', 'a4f4ccd8-33b5-4e52-aa47-acadb648e72f': 'Chr5_A:1657kbp', '2f755348-8dbd-419b-868f-709e06e7d814': 'BES14:60kbp', 'ba1b4351-3750-40f7-88d5-5acd7c8c6b4e': 'Chr8_B:665kbp', '259ebaa5-cddd-4488-962c-1a399cbcc9be': 'Chr8_B:2785kbp', '6f7f0012-6dcf-46c2-95c0-5892300fbdcc': 'Chr11_A:1054kbp', 'e3a55be6-548c-4c29-b6b9-5308ed38a641': 'Chr3_A:542kbp', '03304866-beb7-4e07-8900-4426d3809cea': 'Chr5_A:1378kbp', '087eb211-f674-4630-93d5-07a4787dd64d': 'Chr11_A:1363kbp', '4839039f-7740-4160-a49f-3eb21c4084f4': 'Chr2_B:291kbp', 'c6d6b975-786c-4b75-9f0c-c627ca118f62': 'Chr6_B:1378kbp', '5ab3d5b8-42c6-426d-a4d6-3ba4865e9b8c': 'Chr11_B:652kbp', 'dead5633-767e-487a-89a

## post process the data

In [9]:

ref_dict = {x: y[0] for x, y in zip(ref_names, ref_dev) if y[-1] > 30}
fixed_dict = {x: y[0] for x, y in zip(fixed_names, fixed_dev) if y[-1] > 30}

# filter reads
# remove those where the disrance has not changed
filtered = set()
for read_name, distance in zip(fixed_names, fixed_dev):
    if not read_name in gap_spanning_reads:
       filtered.add(read_name)
    if read_name in gap_spanning_reads and len(gap_spanning_reads[read_name].split()) > 1:
        filtered.add(read_name)
    # if read_name in ref_dict and ref_dict[read_name] == distance:
    #     filtered.add(read_name)
    pass
    
readnames = [n for n in fixed_names if n in ref_dict and n in fixed_dict and not n in filtered]
print(readnames)


# figure out gap groups
gap_groups = set()
for gap in gap_spanning_reads.values():
    if len(gap.split()) == 1:
        gap_groups.add(gap)
gap_colors = {}
gap_scatters = {}
gap_fill_colors = {}
gap_x = {}
gap_y = {}

SCATTER_PALETTE = ["circle", "diamond", "triangle", "square", "inverted_triangle", "plus", "hex", "star", "square_pin", 
                   "triangle_pin"]
COLOR_PALETTE_EXT = COLOR_PALETTE + ["#999999"]


num_groups = len(gap_groups)

if num_groups > len(COLOR_PALETTE_EXT) * len(SCATTER_PALETTE) * 2:
    print("WARNING: not enough colors for all gap groups", num_groups, 
          len(COLOR_PALETTE_EXT) * len(SCATTER_PALETTE) * 2)

for idx, gap in enumerate(gap_groups):
    gap_colors[gap] = COLOR_PALETTE_EXT[idx % len(COLOR_PALETTE_EXT)]
    gap_scatters[gap] = SCATTER_PALETTE[(idx // len(COLOR_PALETTE_EXT)) % len(SCATTER_PALETTE)]
    gap_fill_colors[gap] = gap_colors[gap] if (idx // (len(COLOR_PALETTE_EXT) * len(SCATTER_PALETTE))) % 2 == 0 else None
    gap_x[gap] = []
    gap_y[gap] = []

read_colors = {}
read_fill_colors = {}
read_scatters = {}
for read in readnames:
    read_colors[read] = gap_colors[gap_spanning_reads[read]]
    read_fill_colors[read] = gap_fill_colors[gap_spanning_reads[read]]
    read_scatters[read] = gap_scatters[gap_spanning_reads[read]]
    gap_x[gap_spanning_reads[read]].append(ref_dict[read])
    gap_y[gap_spanning_reads[read]].append(fixed_dict[read])


['ddfef19b-abf2-4826-98d9-da313467b7e7', '81d2e85a-ffc3-4c89-a448-d59ec32747f0', '62d39d2f-805a-4ae3-8c66-4a383d6c4379', '4e69e73a-34a6-4755-8486-cb5168698da8', 'a4f4ccd8-33b5-4e52-aa47-acadb648e72f', 'dead5633-767e-487a-89a2-4ec818a117c9', '1bb12a32-2b56-4a4e-a227-77364c502761', '9c93ae0f-62d2-46ce-892e-2b47610e3373', '9f2be5b3-0236-458b-b4df-888f66e06a47', '9eea2d8c-ce1e-499f-be8d-d10117d2df06', '108a0fb2-bb8a-4a64-a607-bbca8ef798d0', 'f88ef346-8b5e-467c-b6ab-fcaa98d1ece8', '1d4d8f78-16bb-427d-a2d2-e18524e8f7b1', '59088221-50f7-42f2-a2ff-6efdf2ce2406', '372f2fb3-80ec-4bcc-b065-cd0035576ad7', '7c7d2e4f-70ff-4a66-be1d-1b56b0a44dbd', '5ae937f9-0d6d-4005-8af7-242de4685fdd', 'b1bd74aa-9fc5-466b-a9c0-82c82ef9833f', 'c8af1b28-c40c-453a-9569-83846281f485', 'f31626c5-6215-4895-9057-fbeb9cd05316', 'bbf1e2c9-4db1-43a2-ad25-b855d81fb615', 'e501e537-3dae-4009-9b14-63eb7ac5db27', 'f0111760-36dd-4344-b873-8736acc1fef0', '24751b41-1323-4a9b-86dd-a2dd0194a199', '68f7c094-cc0e-4185-bc9e-74191d04661c',

## Plot the results

In [10]:
output_file("../data/out/22_vpr_new_genome/compare.html")


f = figure(x_axis_label="Deviation on reference genome",
        y_axis_label="Deviation on improved genome", 
        tooltips=[("", "@h"), ("", "@r")])
f.line(x=[-1.5*10**5, 0.5*10**5], y=[-1.5*10**5, 0.5*10**5], color="black")

f.scatter(x="x", y="y", line_color="c", fill_color="cf", marker="m", 
        size=9, alpha=0.4, 
        source=ColumnDataSource(data={
                "x": [ref_dict[n] - 1000 for n in readnames], 
                "y": [fixed_dict[n] for n in readnames],
                "c": [read_colors[n] for n in readnames],
                "cf": [read_fill_colors[n] for n in readnames],
                "m": [read_scatters[n] for n in readnames],
                "h": [gap_spanning_reads[n] for n in readnames],
                "r": [n for n in readnames]
            }
                ))

show(f, notebook_handle=True)
save(f)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-d02d7a45-3edc-4af2-b7d1-42cde59ec336.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-d02d7a45-3edc-4af2-b7d1-42cde59ec336.sock'
}


'/ladsie/project/ladsie_019/claudia/ont_assembly_improvement/data/out/22_vpr_new_genome/compare.html'

In [32]:
output_file("../data/out/22_vpr_new_genome/compare2.html")


f = figure(x_axis_label="Deviation on reference genome",
        y_axis_label="Deviation on improved genome", 
        tooltips=[("", "@h")])
f.line(x=[-1.5*10**5, 0.5*10**5], y=[-1.5*10**5, 0.5*10**5], color="black")

def median(lst):
    if len(lst) == 0:
        return None
    return sorted(lst)[len(lst)//2]

def mean(lst):
    if len(lst) == 0:
        return None
    return sum(lst) / len(lst)

# f.scatter(x="x", y="y", line_color="c", fill_color="cf", marker="m", 
#         size=9, alpha=0.1, 
#         source=ColumnDataSource(data={
#                 "x": [ref_dict[n] - 1000 for n in readnames], 
#                 "y": [fixed_dict[n] for n in readnames],
#                 "c": [read_colors[n] for n in readnames],
#                 "cf": [read_fill_colors[n] for n in readnames],
#                 "m": [read_scatters[n] for n in readnames],
#                 "h": [gap_spanning_reads[n] for n in readnames],
#                 "r": [n for n in readnames]
#             }
#                 ))

f.scatter(x="x", y="y", size=9, alpha=0.8, line_color="c", fill_color="cf", marker="m", 
        source=ColumnDataSource(data={
                "x": [mean(gap_x[n]) - 1000 if median(gap_x[n]) is not None else None for n in gap_groups], 
                "y": [mean(gap_y[n]) for n in gap_groups],
                "c": [gap_colors[n] for n in gap_groups],
                "cf": [gap_fill_colors[n] for n in gap_groups],
                "m": [gap_scatters[n] for n in gap_groups],
                "h": [n for n in gap_groups]
            }
                ))

show(f, notebook_handle=True)
save(f)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-d02d7a45-3edc-4af2-b7d1-42cde59ec336.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-d02d7a45-3edc-4af2-b7d1-42cde59ec336.sock'
}


'/ladsie/project/ladsie_019/claudia/ont_assembly_improvement/data/out/22_vpr_new_genome/compare2.html'

## check some individual gaps

In [40]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
# GAP_TO_CHECK = "Chr11_core:263kbp"
#GAP_TO_CHECK = "Chr7_core:1789kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"
# GAP_TO_CHECK = "Chr11_3B:266kbp"
#GAP_TO_CHECK = "Chr6_A:2589kbp"
#GAP_TO_CHECK = "Chr10_B:5341kbp"
# GAP_TO_CHECK = "BES17:65kbp"
# GAP_TO_CHECK = "Chr11_3A:497kbp"
GAP_TO_CHECK = "Chr4_A"

f = figure(title="Comparison of distance deviations", x_axis_label="Reference", y_axis_label="Fixed", 
           tooltips=[("", "@h"), ("", "@r")])

picked_readnames = [n for n in readnames if n in gap_spanning_reads and GAP_TO_CHECK in gap_spanning_reads[n]]

f.line(x=[-1.5*10**5, 0.5*10**5], y=[-1.5*10**5, 0.5*10**5], color="black")
f.scatter(x="x", y="y", line_color=None, fill_color="c", size=9, alpha=0.4, 
          source=ColumnDataSource(data={
                "x": [ref_dict[n] for n in picked_readnames], 
                "y": [fixed_dict[n] for n in picked_readnames],
                "c": [read_colors[n] for n in picked_readnames],
                "h": [gap_spanning_reads[n] for n in picked_readnames],
                "r": picked_readnames
            }
                ))

show(f, notebook_handle=True)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-f98820cc-5cc5-4b7d-a26a-67efeb848081.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-f98820cc-5cc5-4b7d-a26a-67efeb848081.sock'
}


## get read clusters -> turn into table of "correctly" expanded gaps

In [ ]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
#GAP_TO_CHECK = "Chr11_core:263kbp"
#GAP_TO_CHECK = "Chr11_3B:266kbp"
GAP_TO_CHECK = "Chr11_A:4918kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"

f = figure(title="looking at how to best cluster", x_axis_label="Ref fixed difference", y_axis_label="ref fixed sum", 
           tooltips=[("", "@h"), ("", "@r")])

picked_readnames = [n for n in readnames if n in gap_spanning_reads and gap_spanning_reads[n] == GAP_TO_CHECK]

f.scatter(x="x", y="y", line_color=None, fill_color="c", size=9, alpha=0.4, 
          source=ColumnDataSource(data={
                "x": [ref_dict[n] - fixed_dict[n] for n in picked_readnames], 
                "y": [fixed_dict[n] + ref_dict[n] for n in picked_readnames],
                "c": [read_colors[n] for n in picked_readnames],
                "h": [gap_spanning_reads[n] for n in picked_readnames],
                "r": picked_readnames
            }
                ))

show(f, notebook_handle=True)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-e1f9b001-948f-4d16-a626-3f97eb0e0971.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-e1f9b001-948f-4d16-a626-3f97eb0e0971.sock'
}


In [ ]:

def extract_reads_for_gap(gap_name):
    return [n for n in readnames if n in gap_spanning_reads and gap_spanning_reads[n] == gap_name]

def cluster(l, key, max_dif):
    l.sort(key=key)

    clustered = [[l[0]]]
    if len(l) > 1:
        for read in l[1:]:
            if abs(key(read) - key(clustered[-1][-1])) > max_dif:
                clustered.append([])
            clustered[-1].append(read)
    return clustered

def cluster_reads(readnames, max_ref_fixed_diff = 10, max_ref_fixed_sum = 1000):
    return [c for l in cluster(readnames, lambda x: ref_dict[x] - fixed_dict[x], max_ref_fixed_diff) 
               for c in cluster(l, lambda x: ref_dict[x] + fixed_dict[x], max_ref_fixed_sum)]

def filter_clusters(clusters):
    return [c for c in clusters if len(c) > 3]

def get_mean_deviation_in_clusters(clusters, in_fixed=True):
    return [sum([fixed_dict[n] if in_fixed else ref_dict[n] for n in cluster]) / len(cluster) for cluster in clusters]

In [ ]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
#GAP_TO_CHECK = "Chr11_core:263kbp"
GAP_TO_CHECK = "Chr11_A:4918kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"

# picked_readnames = extract_reads_for_gap(GAP_TO_CHECK)


def cluster(l, key, max_dif):
    l.sort(key=key)

    clustered = [[l[0]]]
    if len(l) > 1:
        for read in l[1:]:
            if abs(key(read) - key(clustered[-1][-1])) > max_dif:
                clustered.append([])
            clustered[-1].append(read)
    return clustered

# clustered = cluster_reads(picked_readnames)


# print(clustered)
# print([len(c) for c in clustered])

# clustered = filter_clusters(clustered)

# print(get_mean_deviation_in_clusters(clustered))
# print(get_mean_deviation_in_clusters(clustered, False))


In [ ]:
gap_closed_if_fixed_dev_smaller_than = 5000


gap_names = gap_pos.keys()

closed_gaps = 0
print("correct", "#supp", "#contra", "dev", "fxd_dev", "other", "has_cluster", "name", sep="\t")
with open("../data/in/analysis_in/closed_gaps_analysis.gff", "w") as file_out:
    for gap in sorted(gap_names):
        has_cluster = False #not gap in gap_without_cluster
        read_names = extract_reads_for_gap(gap)
        chrom, start, end = gap_pos[gap]
        if len(read_names) > 0:
            read_clusters = filter_clusters(cluster_reads(read_names))
            cluster_fixed = get_mean_deviation_in_clusters(read_clusters)
            gap_sizes = get_mean_deviation_in_clusters(read_clusters, False)
            gap_closed = False
            gap_idx = 0
            min_fixed = float("inf")
            for idx, x in enumerate(cluster_fixed):
                if abs(x) < gap_closed_if_fixed_dev_smaller_than and abs(x) < min_fixed:
                    gap_closed = True
                    gap_idx = idx
                    min_fixed = abs(x)
            if gap_closed:
                print("Yes", len(read_clusters[idx]), len(read_names) - len(read_clusters[idx]), int(gap_sizes[idx]), 
                    int(cluster_fixed[idx]), len(gap_sizes) > 1, has_cluster, gap, sep="\t")
                closed_gaps += 1
                file_out.write("\t".join([chrom + "_Tb427v10", ".", "fixedgap", str(start), str(end), ".", ".", ".", 
                                          "estimated_length=1000;gap_type=within scaffold;closed_correctly=true"]) + "\n" )
            else:
                print("No" if len(read_names) > 5 else "?", "", len(read_names), "", "", "", has_cluster, gap, sep="\t")
                file_out.write("\t".join([chrom + "_Tb427v10", ".", "notenoughdatagap", str(start), str(end), ".", ".", ".", 
                                          "estimated_length=1000;gap_type=within scaffold;not_enough_data=true"]) + "\n" )
        else:
            print("?", "", "", "", "", "", has_cluster, gap, sep="\t")
            file_out.write("\t".join([chrom + "_Tb427v10", ".", "notenoughdatagap", str(start), str(end), ".", ".", ".", 
                                        "estimated_length=1000;gap_type=within scaffold;not_enough_data=true"]) + "\n" )
print()
print()
print("closed", closed_gaps, "out of", len(gap_names), "gaps")

correct	#supp	#contra	dev	fxd_dev	other	has_cluster	name
Yes	14	2	-1091	-3062	False	False	BES17:65kbp
?		3				False	BES2:62kbp
Yes	21	0	1125	128	False	False	Chr10_A:4101kbp
Yes	14	1	-6040	142	False	False	Chr10_A:76kbp
Yes	11	5	1043	45	False	False	Chr10_B:4083kbp
Yes	10	2	-44272	-464	False	False	Chr10_B:5341kbp
Yes	19	1	-11365	183	False	False	Chr10_B:58kbp
Yes	6	1	-2951	-2951	False	False	Chr11_A:14kbp
?						False	Chr11_A:279kbp
Yes	27	1	-104	82	False	False	Chr11_A:4651kbp
No		25				False	Chr11_A:4918kbp
No		9				False	Chr11_A:4971kbp
?						False	Chr11_B:296kbp
Yes	11	0	1065	69	False	False	Chr11_B:32kbp
Yes	17	3	-10477	48	False	False	Chr11_B:4669kbp
Yes	18	1	-23788	399	False	False	Chr11_B:4955kbp
?		2				False	Chr11_B:5168kbp
?						False	Chr11_B:5455kbp
?		1				False	Chr1_A:100kbp
?						False	Chr1_A:1149kbp
?		1				False	Chr1_A:1212kbp
?		1				False	Chr1_A:2357kbp
Yes	35	2	-3970	116	False	False	Chr1_A:3037kbp
?						False	Chr1_A:711kbp
Yes	9	0	1099	101	False	False	Chr1_A:946kbp
Yes	2